The flag is encrypted using a [Goldwasser-Micali](https://en.wikipedia.org/wiki/Goldwasser%E2%80%93Micali_cryptosystem) like cryptosystem with public key $N$. It can be decrypted given the factorisation of $N$.

The cryptosystem works by taking $N = pq$ to be the product of two large primes $p$ and $q$, and a random value $x$ such that

$$\left ( \dfrac{x}{p} \right ) = \left ( \dfrac{x}{q} \right ) = -1$$

i.e. such that $x$ is a quadratic non-residue modulo $p$ and a quadratic non-residue modulo $q$.

To encrypt a single bit $b$, a random value $1 \leq r < N$ is chosen such that $\gcd(r, N) = 1$, and the value $c \equiv r^2 x^b \pmod N$ is outputted. To encrypt a message, we encrypt all of its bits individually.

Note that if $b = 0$, then the outputted value $c$ will be a quadratic residue modulo $p$, hence, if we find that $c$ is a quadratic non-residue modulo $p$, then we will know that $b = 1$. Similarly, if $b = 1$, then the outputted value $c$ will be a quadratic non-residue modulo $p$. So if we find that $c$ is a quadratic residue modulo $p$, then we will know that $b = 0$.

If we can factorise $N$, we will have enough information to decrypt the flag. We are given a hint that is sufficient for us to solve the factorisation problem.

We are given the value

$$h = D(\lfloor \sqrt p \rfloor + \lfloor \sqrt q \rfloor)$$

where $D = (1 \times 3 \times 3 \times 7)^{1 + 3 + 3 + 7}$ is a known 84 bit constant. (I'll omit the $\lfloor \ \rfloor$ for the rest of the writeup, but keep in mind we're talking about integer square roots)

Note that $p$ and $q$ are not perfect squares (as they are prime), and so it is quite infeasible to recover the actual values of $p$ and $q$ without a bit of work. Taking the integer square root of large numbers removes a lot of information about them such that it is practically impossible to recover the original large number. To understand why, notice that the gap between the $i$th and $i+1$st square numbers is $2i + 1$. e.g. the gap between $7^2$ and $8^2$ is $2 \cdot 7 + 1$. So if we take any integer in the range between these squares (e.g. any integer between 49 and 64 exclusive), then their integer square root will be $7$. This is clearly not a one-to-one mapping, so going from $7$ back to the original number isn't possible without extra information. If we did have extra information that would allow us to check which number is correct, however, we could just enumerate each number in the range. For large numbers (e.g. numbers on the order of 1337 bits), this is computationally infeasible. So is the challenge impossible?

Clearly not! The constant multiple $D$ helps a bit in giving us some extra bits of information, but that alone isn't enough. Since we know some additional information about $p$ and $q$ however, (namely that their product is $n$), there are some techniques we can use to recover them completely.

Similarly to the babyrsa challenge, we can construct a quadratic equation with roots that will help us to find approximations of $p$ and $q$:

$$\begin{aligned}
(x - D\sqrt p)(x - D\sqrt q) &= x^2 - D(\sqrt p + \sqrt q)x + D^2 \sqrt{pq} \\
                                              &= x^2 - hx + D^2 \sqrt n
\end{aligned}$$

Finding a root $x_0$ and calculating $(x_0/D)^2$ will give us an approximation for $p$ (or $q$, it doesn't really matter what we call it). It turns out that this approximation is accurate to around 750 bits on average, so around 590 bits are unknown (we could figure this out by testing locally with our own values). Despite knowing only 750/1337 bits of $p$ however,  we can determine the exact value of $p$ using [Coppersmith's method](https://en.wikipedia.org/wiki/Coppersmith%27s_attack) which finds small roots of polynomials modulo some integer.

Let $p'$ be our approximation for $p$. Then $p' = p + \delta$ where $|\delta| < 2^{590}$. We then construct the monic polynomial

$$\begin{aligned}
    f(x) &\equiv x - p' \pmod n \\
           &\equiv x - p - \delta \pmod n \\
           &\equiv x - \delta \pmod n
\end{aligned}$$

where the last line follows from the fact $a \equiv b \pmod {pq} \implies a \equiv b \pmod p$. So solving for $x$ will give us the value of $\delta$. We have now recovered $p$ and can decrypt the flag as above.

Note that the flag is encrypted slightly different to the standard Goldwasser-Micali encryption. Here, the quadratic non-residue $x$ is raised to the power of $1337 + b$, so when $b = 0$, the result is a quadratic non-residue, and when $b = 1$, the result is a quadratic residue. This just means that if the ciphertext is a quadratic residue, then $b = 1$, and if it is a quadratic non-residue, then $b = 0$. Also, the exponent for the random value $r$ is $1337+1337$, (purely for thematic purposes); it only matters that this value is even.

Solve script:

```python
from Crypto.Util.number import long_to_bytes

exec(open('../challenge/output.txt').read()) # hint, n, c, D

# get an approximation for p
F.<x> = ZZ[]
poly = x^2 - x*hint + D*D*sqrt(n)
d = poly.roots()
p_approx = int((d[0][0]/D)^2)

# compute the exact value of p
P.<x> = PolynomialRing(Zmod(n), implementation='NTL')
f = x + p_approx
d = f.small_roots(X=2**590, beta=0.4, epsilon=1/32)
p = p_approx + d[0]
assert is_prime(p)
print('[+] recovered p', p)

# get the flag
flag = ''.join('0' if kronecker(x, p) == -1 else '1' for x in c)
flag = long_to_bytes(int(flag, 2))
print('[*] flag:', flag.decode())
```